In [20]:
import pandas as pd
import pyodbc

In [21]:
usuario='Netfeira'

senha='sqlserver'

database='MOINHO'

driver='{SQL Server}'

server='192.168.0.252'

str_conexao=('Driver={0};Server={1};Database={4};UID={2};PWD={3}'.format(driver,server,usuario,senha,database))

In [22]:
query="""

WITH TabTpCorte (cd_tp_faltaprd,descrica) AS (

SELECT cd_tp_faltaprd,descricao
FROM tp_faltaprd
WHERE ativo=1
AND descricao LIKE '%CORTE%'),

TabFalta (dt_falta,cd_vend,cd_clien,nu_ped,cd_prod,unid_vda,qtde_falta_vda,preco_unit,vl_total) AS (

SELECT CONVERT(DATETIME,CAST(dt_falta AS date),101) AS dt_falta,
cd_vend,cd_clien,nu_ped,cd_prod,unid_vda,qtde_falta_vda,preco_unit,
ROUND(qtde_falta_vda*preco_unit,2) AS vl_total
FROM faltaprd
INNER JOIN TabTpCorte ON faltaprd.cd_tp_faltaprd=TabTpCorte.cd_tp_faltaprd),

TabSupervisor (cd_equipe,descricao,nome_resumido,email) AS (

SELECT equipe.cd_equipe,descricao,
CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(nome)))=0 THEN nome ELSE
RTRIM(LEFT(nome,(CHARINDEX(' ',LTRIM(RTRIM(nome)))))) + ' ' + 
LTRIM(RIGHT(nome,(CHARINDEX(' ',REVERSE(LTRIM(RTRIM(nome))))))) END AS nome_resumido,
vendedor.e_mail
FROM equipe
INNER JOIN vendedor ON equipe.cd_vend_sup=vendedor.cd_vend
)

SELECT dt_falta AS 'Data de Falta',
TabFalta.cd_vend AS 'ID Vendedor',vendedor.nome_gue AS 'Vendedor',
TabSupervisor.descricao AS 'Equipe',TabSupervisor.nome_resumido AS 'Supervisor',
TabFalta.cd_clien AS 'ID Cliente',cliente.nome_res AS 'Nome Fantasia',
nu_ped AS 'Pedido',TabFalta.cd_prod AS 'SKU',produto.descricao AS 'Produto',
unid_vda AS 'Unid. VDA',qtde_falta_vda AS 'Qtde. VDA',
preco_unit AS 'Valor Unitário',vl_total AS 'Total do Pedido'
FROM TabFalta
INNER JOIN vendedor ON TabFalta.cd_vend=vendedor.cd_vend
INNER JOIN TabSupervisor ON vendedor.cd_equipe=TabSupervisor.cd_equipe
INNER JOIN cliente ON TabFalta.cd_clien=cliente.cd_clien
INNER JOIN produto ON TabFalta.cd_prod=produto.cd_prod
WHERE YEAR(dt_falta)=YEAR(GETDATE()) AND MONTH(dt_falta)=MONTH(GETDATE())
AND DAY(dt_falta)=DAY(GETDate())

"""

In [23]:
def ConexaoSQL():

    try:

        conecta=pyodbc.connect(str_conexao)

        return conecta

        pass

    except:

        print('Sem conexão com a base de dados')

        pass

    pass

In [24]:
def FormatarMoeda(valor):
    
    valor=str(valor)
    
    inteiro=valor[:valor.find('.')]
    
    decimal=valor[valor.find('.'):]
    
    decimal=decimal[1:]
    
    if(len(decimal)==1):
        
        decimal=(f'0{decimal}')
        
        pass
    
    else:
        
        decimal=decimal[:2]
        
        pass
    
    moeda=('R$ {0},{1}'.format(inteiro,decimal))
    
    return moeda
    
    pass

In [31]:
conectando=ConexaoSQL()

df=pd.read_sql(query,conectando)

In [32]:
supervisores=df['Supervisor'].unique().tolist()

for nome in supervisores:
    
    sku=len(df['SKU'].loc[df['Supervisor']==nome].unique().tolist())
    
    pedido=len(df['Pedido'].loc[df['Supervisor']==nome].unique().tolist())
    
    total=df[['Total do Pedido']].loc[df['Supervisor']==nome].sum().tolist()
            
    total=FormatarMoeda(total[0])
    
    mensagem=f"""
    
    Bom dia;
    
    {nome}
    
    Foram identificados cerca de {sku} itens que foram cortados. Totalizando R$ {total}.
    
    """
    
    print(mensagem)
    
    pass


    
    Bom dia;
    
    JULIO DELFINO
    
    Foram identificados cerca de 5 itens que foram cortados. Totalizando R$ R$ 1127,73.
    
    

    
    Bom dia;
    
    ULISSES BACCI
    
    Foram identificados cerca de 4 itens que foram cortados. Totalizando R$ R$ 211,29.
    
    

    
    Bom dia;
    
    DOUGLAS TEDESCHI
    
    Foram identificados cerca de 2 itens que foram cortados. Totalizando R$ R$ 300,02.
    
    

    
    Bom dia;
    
    CARLOS PASSOS
    
    Foram identificados cerca de 3 itens que foram cortados. Totalizando R$ R$ 367,15.
    
    

    
    Bom dia;
    
    JULIO DELFINO
    
    Foram identificados cerca de 5 itens que foram cortados. Totalizando R$ R$ 1127,73.
    
    

    
    Bom dia;
    
    ULISSES BACCI
    
    Foram identificados cerca de 4 itens que foram cortados. Totalizando R$ R$ 211,29.
    
    

    
    Bom dia;
    
    DOUGLAS TEDESCHI
    
    Foram identificados cerca de 2 itens que foram cortados. Totalizando R$ R$ 30

In [33]:
total=df[['Total do Pedido']].loc[df['Supervisor']=='JULIO DELFINO'].sum().tolist()

FormatarMoeda(total[0])

'R$ 1127,73'

'R$ 1127,73'

In [30]:
df.loc[df['Supervisor']=='ULISSES BACCI']

,Data de Falta,ID Vendedor,Vendedor,Equipe,Supervisor,ID Cliente,Nome Fantasia,Pedido,SKU,Produto,Unid. VDA,Qtde. VDA,Valor Unitário,Total do Pedido
2,2022-04-14,VIVIANCR,VIVIAN,EQUIPE 2 (REP),ULISSES BACCI,20721,LUIZ ANTONIO FIGUEIR,391566,611,"IQF MARACUJA C/SEM (1,020KG)",UN,1.0,16.80,16.80
3,2022-04-14,REGIANE,REGIANE DPA,EQUIPE 3 (CLT),ULISSES BACCI,21411,MERCADO VILLAS III,391477,1113,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR),CX,1.0,12.77,12.77
4,2022-04-14,LEOLOREN,LEONARDO,EQUIPE 3 (CLT),ULISSES BACCI,15446,PANIFICADORA SAO PAU,391521,1058,"CONG. BATATA MCCAIN SMILES NAC (1,5KG)",UN,6.0,26.54,159.24
5,2022-04-14,RONALDBU,RONALDO,EQUIPE 3 (CLT),ULISSES BACCI,24379,SUPER ROSSI LJ13,390532,972,IQF FRUTAS VERMELHAS CONGELADA POTE 450G,CX,1.0,22.48,22.48


,Data de Falta,ID Vendedor,Vendedor,Equipe,Supervisor,ID Cliente,Nome Fantasia,Pedido,SKU,Produto,Unid. VDA,Qtde. VDA,Valor Unitário,Total do Pedido
2,2022-04-14,VIVIANCR,VIVIAN,EQUIPE 2 (REP),ULISSES BACCI,20721,LUIZ ANTONIO FIGUEIR,391566,611,"IQF MARACUJA C/SEM (1,020KG)",UN,1.0,16.80,16.80
3,2022-04-14,REGIANE,REGIANE DPA,EQUIPE 3 (CLT),ULISSES BACCI,21411,MERCADO VILLAS III,391477,1113,CONG. BATATA MCCAIN 9MM TRAD 720G (12X720GR),CX,1.0,12.77,12.77
4,2022-04-14,LEOLOREN,LEONARDO,EQUIPE 3 (CLT),ULISSES BACCI,15446,PANIFICADORA SAO PAU,391521,1058,"CONG. BATATA MCCAIN SMILES NAC (1,5KG)",UN,6.0,26.54,159.24
5,2022-04-14,RONALDBU,RONALDO,EQUIPE 3 (CLT),ULISSES BACCI,24379,SUPER ROSSI LJ13,390532,972,IQF FRUTAS VERMELHAS CONGELADA POTE 450G,CX,1.0,22.48,22.48
